In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

block_size = 8 #
batch_size = 4 #
max_iterations = 1000 #
learning_rate = 3e-4 #
evaluate_iterations = 250 #
n_embd = 384
n_head = 8
embedding_vector = [0.1, 0.2, 0.3]
n_layer = 8
dropout = 0.2

Using device: cpu


In [28]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
chars =  sorted(set(text))

vocabulary_size = len(chars)

print(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [29]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([91, 48, 66, 77, 69, 62, 26,  1, 32, 72, 75, 72, 77, 65, 82,  1, 58, 71,
        61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,  1, 66, 71,  1, 43, 83,
         0,  0, 29, 78, 77, 65, 72, 75, 26,  1, 40, 14,  1, 34, 75, 58, 71, 68,
         1, 30, 58, 78, 70,  0,  0, 37, 69, 69, 78, 76, 77, 75, 58, 77, 72, 75,
        26,  1, 38, 72, 65, 71,  1, 46, 14,  1, 42, 62, 66, 69, 69,  0,  0, 46,
        62, 69, 62, 58, 76, 62,  1, 61, 58, 77])


In [30]:
# split the data into train and validation sets
n=int(0.8*len(data)) #0.8
train_data = data[:n]
val_data = data[n:]

# function to generate a batch of data
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    x,y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x.shape)
print(x)
print('targets:')
print(y.shape)
print(y)

'''
https://www.youtube.com/watch?v=BTVUPhpXUd4&list=PLOZ8y37SEZTbf_57K1I4g7Jjrr6raTxbk&index=5
https://www.youtube.com/watch?v=Ip6NkFQ5DKw
https://www.youtube.com/watch?v=2FYzX2eW_aw

'''

inputs:
torch.Size([4, 8])
tensor([[69, 76, 72,  1, 73, 69, 58, 60],
        [71, 69, 82,  1, 77, 65, 66, 71],
        [76, 72, 70, 62,  1, 72, 63,  1],
        [78, 71, 61, 75, 62, 61,  1, 77]])
targets:
torch.Size([4, 8])
tensor([[76, 72,  1, 73, 69, 58, 60, 62],
        [69, 82,  1, 77, 65, 66, 71, 64],
        [72, 70, 62,  1, 72, 63,  1, 72],
        [71, 61, 75, 62, 61,  1, 77, 72]])


'\nhttps://www.youtube.com/watch?v=BTVUPhpXUd4&list=PLOZ8y37SEZTbf_57K1I4g7Jjrr6raTxbk&index=5\nhttps://www.youtube.com/watch?v=Ip6NkFQ5DKw\nhttps://www.youtube.com/watch?v=2FYzX2eW_aw\n\n'

In [31]:
class Head(nn.Module):
    '''one head of self-attention'''

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # lower triangular matrix
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1] **-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out 

class FeedForward(nn.Module):
    ''' a simple linear layer followed by a non-linearity '''

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):

    ''' transformer block communication follwed by computation '''

    def __init__(self, n_embd, n_head):
        super().__init__()

        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.lnl = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        y = self.sa(x)
        x = self.lnl(x + y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x

In [32]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])

        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):

        B, T = index.shape

        logits = self.token_embedding_table(index)  # (B,T,C)

        token_embeddings = self.token_embedding_table(index)  # (B,T,C)
        position_embeddings = self.position_embedding_table(torch.arange(T, device = device))
        x = token_embeddings + position_embeddings
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :] / 0.8
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLanguageModel(vocabulary_size)
m = model.to(device)
print(f"Model parameters: {sum(p.numel() for p in m.parameters()):,}")

# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)


Model parameters: 14,261,084


In [33]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(evaluate_iterations)
        for k in range(evaluate_iterations):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [34]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iterations):
    if iter % evaluate_iterations == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())



step 0: train loss 4.6426, val loss 4.6510
step 250: train loss 2.6076, val loss 2.7824
step 500: train loss 2.5173, val loss 2.8031
step 750: train loss 2.4893, val loss 2.6585
3.0518765449523926


In [ ]:
# context = torch.zeros((1,1), dtype = torch.long,  device = device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)


# prompt = 'Hello! Can you see me?'
# context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context.unsqueeze(0), 100))
# print(generated_chars)

IndexError: index out of range in self